In [2]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../dataset/mnist", one_hot=True)

Extracting ../dataset/mnist/train-images-idx3-ubyte.gz
Extracting ../dataset/mnist/train-labels-idx1-ubyte.gz
Extracting ../dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting ../dataset/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
# 训练参数
learning_rate = 0.001
num_steps = 200
batch_size = 128
display_step = 10

# 网络参数
num_input = 784 # Mnist图像输入尺寸为 28x28=784
num_classes = 10 # Mnist分类为0~9共10类
dropout = 0.75 # Dropout保留神经元的能力

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [6]:
def conv2d(x, W, b, strides=1):
    # SAME padding 给平面外部补零，使得到卷积后得到的平面和原来一样大
    # VALID padding 不会超出平面外部，卷积之后会得到一个比原来小的平面
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpooling2d(x, k=2):
    return tf.nn.maxpooling2d(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                             padding='SAME')

def conv_net(x, weight, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # 卷积层
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # 最大池化层
    conv1 = maxpool2d(conv1, k=2)
    
    # 卷积层
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # 最大池化
    conv2 = maxpool2d(conv2, k=2)
    
    # 全连接层
    # 将 conv2 的输出尺寸与全连接层匹配
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # 使用 dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # 输出,分类预测
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# 用于保存网络层的 weight 和 bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# 构建模型
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# 定义 loss 和 optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 127433.1094, Training Accuracy= 0.109
Step 10, Minibatch Loss= 32216.0469, Training Accuracy= 0.188
Step 20, Minibatch Loss= 16806.7852, Training Accuracy= 0.336
Step 30, Minibatch Loss= 9415.0068, Training Accuracy= 0.633
Step 40, Minibatch Loss= 4466.2749, Training Accuracy= 0.711
Step 50, Minibatch Loss= 5437.2803, Training Accuracy= 0.727
Step 60, Minibatch Loss= 4500.9922, Training Accuracy= 0.727
Step 70, Minibatch Loss= 3532.2788, Training Accuracy= 0.797
Step 80, Minibatch Loss= 4087.4521, Training Accuracy= 0.812
Step 90, Minibatch Loss= 3151.7241, Training Accuracy= 0.844
Step 100, Minibatch Loss= 2683.4912, Training Accuracy= 0.812
Step 110, Minibatch Loss= 1417.4457, Training Accuracy= 0.898
Step 120, Minibatch Loss= 1595.9890, Training Accuracy= 0.906
Step 130, Minibatch Loss= 1553.0103, Training Accuracy= 0.891
Step 140, Minibatch Loss= 1202.6234, Training Accuracy= 0.898
Step 150, Minibatch Loss= 2053.0991, Training Accuracy= 0.859
Step 160, Minib